In [ ]:
# Install library not packaged in Fabric
%pip install ratelimit

StatementMeta(, ccb74f7b-1bf0-4b4e-be32-c76b574ffa33, 11, Finished, Available, Finished)

  Preparing metadata (setup.py) ... - \ done
  Created wheel for ratelimit: filename=ratelimit-2.2.1-py3-none-any.whl size=5894 sha256=781ca4f7ab87b4ebf05e6d9ae448c6019bf452bdf702fad3d57001700e182cf6
  Stored in directory: /home/trusted-service-user/.cache/pip/wheels/ee/d5/e5/8fbffe089140fb498987b7709becf861086daace105d243475
Successfully built ratelimit

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [ ]:
import requests
import pandas as pd
from datetime import datetime, timezone
import os
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from unidecode import unidecode
from threading import Lock
from ratelimit import limits, sleep_and_retry
from pyspark.sql import SparkSession
from py4j.java_gateway import java_import
import time

spark = SparkSession.builder.getOrCreate()

# Config
range_hs2 = range(1, 100)   # Dimension 5 - HS2 (Product Code)
range_states = range(1, 55) # Dimension 3 - US States
n_periods = 60  # last 5 years of monthly data
post_url = "https://www150.statcan.gc.ca/t1/wds/rest/getDataFromCubePidCoordAndLatestNPeriods"
lakehouse_path = "abfss://1dcd65a7-d5a3-4e2b-a110-db438703b7b5@onelake.dfs.fabric.microsoft.com/f8c35c71-7fa4-4e87-9b9a-91435e298fb7/Tables"
product_ids = {
    "exports": 12100147, # Canadian international merchandise trade data cube product IDs
    "imports": 12100150
}

# Check if lakehouse files exist
def table_exists(table_name):
    try:
        spark.table(table_name)
        return True
    except:
        return False

# Retrieve and process JSON
@sleep_and_retry
@limits(calls=1, period=1) # Limit to one call per second
def get_recent_data(post_url: str, post_body: list, extra_col: tuple):
    trade_type = extra_col[3]
    product_id = post_body[0]["productId"]

    # Make API request, extract JSON file
    session = get_session_with_retries()
    response = session.post(post_url, json=post_body, timeout=(5, 30))
    response.raise_for_status()
    json_data = response.json()

    # List indexing and key searching for records
    records = json_data[0]["object"]["vectorDataPoint"] 

    # Add additional re-exports column for the exports file
    if trade_type == "exports":
        reexports_records = get_reexport(product_id, extra_col[2]) # Pull JSON with values for reexports instead of domestic exports
        
        # Populate domestic exports JSON with a reexport field
        if ((len(reexports_records) == len(records)) and (len(reexports_records) != 0) and (len(records) != 0)):
            for i in range(0, len(records)):
                value = reexports_records[i]["value"]
                records[i]["reexport_value"] = int(value) if value is not None else 0

        # Populate reexports with 0 values if HS2/State combination does not have reexports but has domestic exports 
        elif ((len(records) > len(reexports_records)) and (len(reexports_records) == 0)):
            print("No existing re-export data for coordinate ", post_body[0]["coordinate"], " (Domestic: ", len(records), " - Re-export: ", len(reexports_records), ")")  
            for record in records:
                record["reexport_value"] = 0

        # Populate domestic exports with 0 values if HS2/State combination does not have domestic exports but has reexports
        elif ((len(reexports_records) > len(records)) and (len(records) == 0)):
            print("No existing domestic export data for coordinate ", post_body[0]["coordinate"], " (Domestic: ", len(records), " - Re-export: ", len(reexports_records), ")")  
            records = reexports_records
            for record in records:
                record["reexport_value"] = record["value"]
                record["value"] = 0

        else:
            print("Differing amount of export data for domestic exports and re-exports for coordinate - ", post_body[0]["coordinate"], " (Domestic: ", len(records), " - Re-export: ", len(reexports_records), ")")  
    

    # Convert to DataFrame, rename value column as suite
    df = pd.DataFrame.from_records(records)
    
    if trade_type == "exports":
        df.rename(columns={'value': 'domestic_export_value'}, inplace=True)
        value_col = "domestic_export_value"
    elif trade_type == "imports":
        df.rename(columns={'value': 'import_value'}, inplace=True)
        value_col = "import_value"
    else:
        value_col = "value"  # Fallback

    # Fill NaNs and cast to int safely
    for col in [value_col, "reexport_value"]:
        if col in df.columns:
            df[col] = df[col].fillna(0).astype(int)

    # Rename value column name for better representation
    if trade_type == "exports":
        df.rename(columns={'value': 'domestic_export_value'}, inplace=True)
    elif trade_type == "imports":
        df.rename(columns={'value': 'import_value'}, inplace=True)

    # Enrich with additional columns for info/metadata
    try:
        df["principal_trading_partner_state"] = pull_values(extra_col[0], "State", product_id)
        df["hs2_commodity"] = unidecode(pull_values(extra_col[1], "HS2", product_id))
        df["coordinate"] = extra_col[2]
        # df["product_id"] = post_body[0]["productId"]
        df["retrieved_timestamp"] = datetime.now(timezone.utc).isoformat()
    except AttributeError:
        print(f"No values for {extra_col} found for coordinate {extra_col[2]}")
    return df
    
#---------------------------------------------------------------------------#

# Pull reexport values for the exports table
def get_reexport(pid: int, coords: str):
    post_url = "https://www150.statcan.gc.ca/t1/wds/rest/getDataFromCubePidCoordAndLatestNPeriods"

    # Takes coordinate for the domestic exports of the HS2/State, turns the coordinate into its respective reexport version
    coords_split = coords.split(".")
    coords_split[3] = "2"
    new_coords = ".".join(coords_split)

    # Make API request, extract JSON file
    post_body = [{"productId": pid, "coordinate": new_coords, "latestN": n_periods}]
    session = get_session_with_retries()
    response = session.post(post_url, json=post_body, timeout=(5, 30))
    reexport_data = response.json()

    # List indexing and key searching for records
    reexport_output = reexport_data[0]["object"]["vectorDataPoint"]
    
    return reexport_output

#---------------------------------------------------------------------------#

# Pull the name of the State/HS2 by referencing metadata
metadata_lock = Lock()
def pull_values(member_id: int, field: str, product_id: int):
    with metadata_lock:
        # Pulls JSON of metadata if not pulled already
        if product_id not in metadata_cache:
            url = "https://www150.statcan.gc.ca/t1/wds/rest/getCubeMetadata"
            post_body = [{"productId":product_id}]

            response = requests.post(url, json=post_body)
            data = response.json()
            metadata_cache[product_id] = response.json()

    # Returns HS2 commodity or state, depending on what is requested
    data = metadata_cache[product_id]
    for dimension in data[0]["object"]["dimension"]:
        if dimension["dimensionNameEn"] == field:
            for member in dimension["member"]:
                if member["memberId"] == member_id:
                    return member["memberNameEn"]
    
    return None

#---------------------------------------------------------------------------#

# Retry an API request in case of failure (adds a delay in case of rate limiting)
def get_session_with_retries():
    session = requests.Session()
    retries = Retry(
        total = 5,
        backoff_factor = 2, # exponential backoff: 1s, 2s, 4s, etc.
        status_forcelist = [429, 502, 503, 504],
        allowed_methods = ["POST"],
        raise_on_status = False
    )
    adapter = HTTPAdapter(max_retries=retries)
    session.mount("https://", adapter)
    return session

#---------------------------------------------------------------------------#

write_lock = Lock()

# Call get_recent_data() to get the DataFrame for inputted coordinate, appends to lakehouse table
def process_request(trade_type, pid, state, hs2):
    # Set coordinates based on input
    coordinates = {
        "exports": f"35.9.{state}.1.{hs2}.0.0.0.0.0",
        "imports": f"35.9.{state}.1.{hs2}.0.0.0.0.0"
    }

    post_body = [{"productId": pid, "coordinate": coordinates[trade_type], "latestN": n_periods}]
    df = get_recent_data(post_url, post_body, (state, hs2, coordinates[trade_type], trade_type))
    output_path = f"{lakehouse_path}/bronze_hs2_ontario_us_trade_{trade_type}_data"

    with write_lock:
        # If not data was extracted for a specific coordinate, appends coordinate to a list to try and call late
        if df.empty and hs2 != 77: # No HS2 77 code
            print(f"No data for {trade_type}, state {state}, HS2 {hs2}")
            missed_coords.append((trade_type, pid, state, hs2))
            return "Error"

        # Turn pandas DataFrame into a spark DataFrame
        else:
            try:
                spark_df = spark.createDataFrame(df)
            except Exception as e:
                print(f"Error converting [{trade_type} - State: {state} - {hs2}] to DataFrame: {e}")
                return "Error"

        table_name = f"bronze_hs2_ontario_us_trade_{trade_type}_data"

        # Appends data to table, or creates a new one if table does not exist
        if table_exists(table_name):
            spark_df.write.format("delta").option("mergeSchema", "true").mode("append").saveAsTable(table_name)
        else:
            spark_df.write.format("delta").option("overwriteSchema", "true").mode("overwrite").saveAsTable(table_name)

    return f"Successfully saved {trade_type} data (State: {state}, HS2: {hs2}) to {output_path}"

#---------------------------------------------------------------------------#

# Retrieve and filter for Ontario-U.S.
if __name__ == "__main__":
    metadata_cache = {}
    written_files = set()
    missed_coords = []

    start_time = time.time()

    # Multithreading API calls for parallel calls
    with ThreadPoolExecutor(max_workers=2) as executor:
        futures = []

        # Iterate through coordinates for different State/HS2 for imports and exports
        for trade_type, pid in product_ids.items():
            for state in range_states:
                for hs2 in range_hs2:
                    futures.append(executor.submit(process_request, trade_type, pid, state, hs2))
                    
        for future in as_completed(futures):
            print(future.result())
    
        print(f"Coordinates Not Written in Initial Call: {missed_coords}")
        
        # Recall any coordinates that did not produce an output
        missed_coords_copy = list(missed_coords)

        for coord in missed_coords_copy:
            status = process_request(coord[0], coord[1], coord[2], coord[3]) 
           
    end_time = time.time()
    time_elapsed = int(end_time - start_time)

    print(f"Completed in {time_elapsed} second(s)")


StatementMeta(, ccb74f7b-1bf0-4b4e-be32-c76b574ffa33, 13, Submitted, Running, Running)

Successfully saved exports data (State: 1, HS2: 2) to abfss://1dcd65a7-d5a3-4e2b-a110-db438703b7b5@onelake.dfs.fabric.microsoft.com/f8c35c71-7fa4-4e87-9b9a-91435e298fb7/Tables/bronze_hs2_ontario_us_trade_exports_data
Successfully saved exports data (State: 1, HS2: 1) to abfss://1dcd65a7-d5a3-4e2b-a110-db438703b7b5@onelake.dfs.fabric.microsoft.com/f8c35c71-7fa4-4e87-9b9a-91435e298fb7/Tables/bronze_hs2_ontario_us_trade_exports_data
No existing re-export data for coordinate  35.9.1.1.4.0.0.0.0.0  (Domestic:  60  - Re-export:  0 )
Successfully saved exports data (State: 1, HS2: 3) to abfss://1dcd65a7-d5a3-4e2b-a110-db438703b7b5@onelake.dfs.fabric.microsoft.com/f8c35c71-7fa4-4e87-9b9a-91435e298fb7/Tables/bronze_hs2_ontario_us_trade_exports_data
Successfully saved exports data (State: 1, HS2: 4) to abfss://1dcd65a7-d5a3-4e2b-a110-db438703b7b5@onelake.dfs.fabric.microsoft.com/f8c35c71-7fa4-4e87-9b9a-91435e298fb7/Tables/bronze_hs2_ontario_us_trade_exports_data
Successfully saved exports data (